Домашнее задание по занятию №7.
Задание №2.
***
Реализовать с использованием потоков и процессов скачивание файлов из интернета. 
Список файлов для скачивания подготовить самостоятельно (например изображений, не менее 100 изображений или других объектов). Сравнить производительность с последовательным методом. Сравнить производительность Thread и multiprocessing решений. Попробовать подобрать оптимальное число потоков/процессов. 

***
Анализ задания и алгоритм выполнения.
Находим простую страницу в интернете с картинками, имеющими url адреса (вместо выдачи с помощью JavaScript)
Скачиваем страницу, анализируем ее на наличие url адресов картинок.
Адреса картинок помещаем в список.
Затем загружаем картинки по списку:
а)  последовательно одну за другой
*) с помощью Thread
*) с помощью multiprocessing
В каждом случае засекаем время скачивания
 

In [1]:
import requests
import re
import os
import time
import urllib.request
from concurrent.futures import ThreadPoolExecutor, ProcessPoolExecutor


# Function to extract a list of image urls from a text of an internet page
# page - the html page to extract urls of images from
# protocol - the protocol of the website like "http" or "https"
#
def get_img_url_in_text(page, protocol):
    urls = []
    all_urls = re.findall(r'((http\:|https\:)?\/\/[^"\' ]*?\.(png|jpg))', page, flags=re.IGNORECASE | re.MULTILINE | re.UNICODE)
    for url in all_urls:
        if not url[0].startswith("http"):
            urls.append(protocol + url[0])
        else:
            urls.append(url[0])
    return urls

# Function to download a page and then evoke the function above to extract image urls:
#
def get_images_from_url(url):
    protocol = url.split('/')[0]
    try:
        resp = requests.get(url)
        return get_img_url_in_text(resp.text, protocol)
    except:
        print("Error while the page loading\nNo image address can be extracted")
      

Добываем список адресов картинок из задаваемого нами url. При этом скачивается страница,  после чего анализируется на наличие ссылок на картинки.
Ссылки извлекаются в список.

In [2]:
url = "https://www.theshermantank.com/sherman/45-gallery-2-more-random-high-resolution-sherman-photos-with-comments/"
# url = "https://www.theshermantank.com/"
image_list = get_images_from_url(url)

Выведем адреса первых десяти картинок:

In [3]:
print('URLs of several pictures:')
i = 0
while i < 10 and i < len(image_list):
    print(image_list[i])
    i += 1
print(f'Total number of images found in the page: {len(image_list)}')

# extracting addresses for 100 images
if len(image_list) > 16:
    image_list = image_list[0:16]


URLs of several pictures:
https://www.theshermantank.com/wp-content/uploads/sherman-tank-site-top-1.jpg
https://www.theshermantank.com/wp-content/uploads/2015/11/cropped-AnnoM4A3E8_Sherman-32x32.png
https://www.theshermantank.com/wp-content/uploads/2015/11/cropped-AnnoM4A3E8_Sherman-192x192.png
https://www.theshermantank.com/wp-content/uploads/2015/11/cropped-AnnoM4A3E8_Sherman-180x180.png
https://www.theshermantank.com/wp-content/uploads/2015/11/cropped-AnnoM4A3E8_Sherman-270x270.png
https://www.theshermantank.com/wp-content/uploads/M4Sherman_M1dozerblade_1945.jpg
https://www.theshermantank.com/wp-content/uploads/M4Sherman_M1dozerblade_1945.jpg
https://www.theshermantank.com/wp-content/uploads/M4Sherman_M1dozerblade_1945.jpg
https://www.theshermantank.com/wp-content/uploads/M4Sherman_M1dozerblade_1945-300x228.jpg
https://www.theshermantank.com/wp-content/uploads/M4_Sherman_105_1944-1.jpg
Total number of images found in the page: 289


Видим, что страница включает 289 картинок.

Скачивание картинок последовательно одной за другой, без распараллеливания. 
Засекаем время и выводим на экран.

In [4]:
dir = os.getcwd() + "/" + "images" # folder for images to be downloaded
time1 = time.perf_counter() # starting the timer

for image_address in image_list:
    try:
        filename = dir + "/" + image_address.split('/')[-1] # filename includes the folder and the original picture name
        print(filename)
        urllib.request.urlretrieve(image_address, filename) # downloading the page        
    except:
        print(f"Error downloading file")

time2 = time.perf_counter() # stopping the timer
time_sequential = time2-time1
print(f'Time of sequential download of the pictures: {time_sequential} s')

d:\CurrentWork\ML\Lesson07_Python_decorators_threads\Homework_07/images/sherman-tank-site-top-1.jpg
d:\CurrentWork\ML\Lesson07_Python_decorators_threads\Homework_07/images/cropped-AnnoM4A3E8_Sherman-32x32.png
d:\CurrentWork\ML\Lesson07_Python_decorators_threads\Homework_07/images/cropped-AnnoM4A3E8_Sherman-192x192.png
d:\CurrentWork\ML\Lesson07_Python_decorators_threads\Homework_07/images/cropped-AnnoM4A3E8_Sherman-180x180.png
d:\CurrentWork\ML\Lesson07_Python_decorators_threads\Homework_07/images/cropped-AnnoM4A3E8_Sherman-270x270.png
d:\CurrentWork\ML\Lesson07_Python_decorators_threads\Homework_07/images/M4Sherman_M1dozerblade_1945.jpg
d:\CurrentWork\ML\Lesson07_Python_decorators_threads\Homework_07/images/M4Sherman_M1dozerblade_1945.jpg
d:\CurrentWork\ML\Lesson07_Python_decorators_threads\Homework_07/images/M4Sherman_M1dozerblade_1945.jpg
d:\CurrentWork\ML\Lesson07_Python_decorators_threads\Homework_07/images/M4Sherman_M1dozerblade_1945-300x228.jpg
d:\CurrentWork\ML\Lesson07_Python_

Скачивание картинок с помощью потоков (Threads)

In [ ]:
time1 = time.perf_counter()

with ThreadPoolExecutor(max_workers = 16) as exec:
        futures = []
        for image_address in image_list:
            try:
                filename = dir + "/" + image_address.split('/')[-1] # filename includes the folder and the original picture name
                # print(filename)                
                futures.append(exec.submit(urllib.request.urlretrieve, image_address, filename)) # downloading the page in threads
            except:
                 print('Image download fail')

time2 = time.perf_counter() # stopping the timer
time_multithread = time2-time1
print(f'Time of multi-threaded download of the pictures: {time_multithread} s')

d:\CurrentWork\ML\Lesson07_Python_decorators_threads\Homework_07/images/sherman-tank-site-top-1.jpg
d:\CurrentWork\ML\Lesson07_Python_decorators_threads\Homework_07/images/cropped-AnnoM4A3E8_Sherman-32x32.png
d:\CurrentWork\ML\Lesson07_Python_decorators_threads\Homework_07/images/cropped-AnnoM4A3E8_Sherman-192x192.png
d:\CurrentWork\ML\Lesson07_Python_decorators_threads\Homework_07/images/cropped-AnnoM4A3E8_Sherman-180x180.png
d:\CurrentWork\ML\Lesson07_Python_decorators_threads\Homework_07/images/cropped-AnnoM4A3E8_Sherman-270x270.png
d:\CurrentWork\ML\Lesson07_Python_decorators_threads\Homework_07/images/M4Sherman_M1dozerblade_1945.jpg
d:\CurrentWork\ML\Lesson07_Python_decorators_threads\Homework_07/images/M4Sherman_M1dozerblade_1945.jpg
d:\CurrentWork\ML\Lesson07_Python_decorators_threads\Homework_07/images/M4Sherman_M1dozerblade_1945.jpg
d:\CurrentWork\ML\Lesson07_Python_decorators_threads\Homework_07/images/M4Sherman_M1dozerblade_1945-300x228.jpg
d:\CurrentWork\ML\Lesson07_Python_

Скачивание картинок с помощью параллельных процессов (multiprocessing)

In [21]:
time1 = time.perf_counter()

with ProcessPoolExecutor(max_workers = 16) as exec:
        futures = []
        for image_address in image_list:
            try:
                filename = dir + "/" + image_address.split('/')[-1] # filename includes the folder and the original picture name
                # print(filename)                
                futures.append(exec.submit(urllib.request.urlretrieve, image_address, filename)) # downloading the page in threads
            except:
                 print('Image download fail')

time2 = time.perf_counter() # stopping the timer
time_multiprocess = time2-time1
print(f'Time of multi-process download of the pictures: {time_multiprocess} s')

Time of multi-process download of the pictures: 22.07577499999752 s


Сравниваем время скачивания тремя методами:

In [22]:
print(f"Time for sequential download: {time_sequential}")
print(f"Time for multithread download: {time_multithread}")
print(f"Time for multiprocess download: {time_multiprocess}")
print(f"Benefit from multithread download vs sequential, %: {(time_sequential - time_multithread) / time_sequential *100}")
print(f"Benefit from multiprocess download vs sequential, %: {(time_sequential - time_multiprocess) / time_sequential *100}")

Time for sequential download: 165.61670810000214
Time for multithread download: 22.15297759999885
Time for multiprocess download: 22.07577499999752
Benefit from multithread download vs sequential, %: 86.62394763539044
Benefit from multiprocess download vs sequential, %: 86.67056285971594


Выводы:
В ходе выполнения задания реализовано скачивание набора картинок с интернет-страницы тремя методами: 
    ●   Последовательный метод
    ●   Параллельными потоками
    ●   Параллельными процессами
В качестве замечания отметим, что очень большое время ушло на поиск подходящей web-страницы со статическими адресами картинок. Большинство страниц с динамически формируемым содержимым не позволяют извлечь адреса картинок для последующего скачивания.
В ходе работы было показано, что использование параллельной обработки позволяет ускорить скачивание набора файлов.
Ускорение от перехода к параллельному скачиванию в нашем случае позволяет сократить время скачивания примерно на порядок (иногда чуть больше чем в 10 раз, иногда меньше). При этом дополнительная сложность для оценки требуемого времени скачивания вносится нестабильностью скорости интернет-соединения.
Для оценки оптимального числа потоков мы использовали 16 файлов вместо 100 в связи с большими затратами времени на такие эксперименты. Если при последовательном скачивании 16 файлов потребовалось ~180 с, то параллельные потоки позволяют уменьшить это время: для 2 потоков временные затраты составляют уже ~80-100 c, и далее монотонно падают до ~16-22 с при увеличении числа потоков до 16. По всей видимости, должен существовать оптимум числа потоков, после которого дальшейшее увеличение их числа уже не будет давать выигрыша и даже приводить к увеличению требуемого времени скачивания (кто застал эпоху масс-даунлоадеров наподобие Download Master, знает об этом на собственном опыте). Иногда (хоть и не всегда) выход на плато по времени скачивания наблюдается уже начиная с 12 потоков.
Отметим, что измерения проводились на четыреъядерном процессоре с hyperthreading, то есть имеющем 8 аппаратных потоков выполнения.
Что касается влияния числа процессов на скорость скачивания, то из-за упомянутой нестабильности результатов сложно оценить, насколько различный результат дает многопоточная и многопроцессная обработка потоков скачивания. В целом, в условиях нашего эксперимента наблюдалось более-менее эквивалентное поведение: при увеличении числа процессов время скачивания падает, при равном числе потоков и процессов время более-менее одинаковое (с учетом нестабильности интернет-соединения). Иногда наблюдалось, что использование параллельных потоков дает больший выигрыш, чем использование параллельных процессов, иногда наоборот. С одной стороны, из-за накладных расходов времени, требуемого для запуска каждого последующего процесса, использование параллельных потоков может быть выгоднее параллельных процессов. С другой стороны, независимое выполнение процесоов на разных ядрах процессора может дать преимущество именно для обработки с помощью параллельных процессов. 
в целом можно сделать вывод, что в условиях нашего эксперимента увеличение числа потоков/процессов до 12-16 является целесообразным.